In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import searoute as sr
import ast
import plotly.graph_objects as go
from copy import deepcopy
import time
from sklearn.cluster import KMeans
from __future__ import annotations
from typing import Dict, Set, Tuple
from dataclasses import dataclass
import random
from plotly.subplots import make_subplots
from collections import defaultdict, deque
import math
import os
import pandas as pd

### TO RUN ON EVERY MACHINE


In [ ]:
import gurobipy as gp

options = {
    "WLSACCESSID": "825a5a39-a867-41e7-b16e-ca6ab5****",
    "WLSSECRET": "d764b4aa-456a-4aa8-90aa-570a98ec****",
    "LICENSEID": 266****,
}


### HELPER FUNCTIONS AND DATA READING


In [ ]:
def make_arc_revenue(
    distances: dict[tuple[int, int], float],
    vessel_capacity: float,
    freight_rate_df: pd.DataFrame,
    trade_data: pd.DataFrame
) -> dict[tuple[int, int, int], float]:
    """
    Compute expected revenue for each origin–destination–commodity arc.

    For each (origin_port, dest_port, commodity) in `trade_data`, if a distance exists in `distances`,
    look up the corresponding freight rate from `freight_rate_df` and scale it by the vessel capacity.

    The freight rate is determined by linear interpolation:
      • If actual distance ≤ 1000 nm: use "FR low"
      • If actual distance ≥ reference distance (Distance_nm): use "FR High"
      • Otherwise: interpolate between "FR low" and "FR High" proportionally.

    Args:
        distances: Mapping (origin_port_id, dest_port_id) → distance in nautical miles.
        vessel_capacity: Cargo capacity of the vessel (e.g., in tonnes).
        freight_rate_df: DataFrame with columns:
            - "HS6"          : commodity code (integer)
            - "Distance_nm"  : reference distance (float)
            - "FR low"       : rate at or below 1000 nm (float, $/tonne)
            - "FR High"      : rate at or above reference distance (float, $/tonne)
        trade_data: DataFrame with at least columns:
            - "origin_port_id" : origin port identifier (int)
            - "dest_port_id"   : destination port identifier (int)
            - "k"              : commodity code (int)

    Returns:
        A dictionary mapping (origin_port_id, dest_port_id, commodity) → revenue ($),
        computed as `chosen_rate * vessel_capacity`. Arcs for which no freight-rate entry exists
        or no distance is found are omitted.
    """
    # Pre-build a lookup: commodity_code → (ref_distance, fr_low, fr_high)
    # This avoids filtering the DataFrame for each row in trade_data.
    rate_lookup: dict[int, tuple[float, float, float]] = {}
    for _, row in freight_rate_df.iterrows():
        hs6 = int(row["HS6"])
        rate_lookup[hs6] = (
            float(row["Distance_nm"]),
            float(row["FR low"]),
            float(row["FR High"])
        )

    revenue: dict[tuple[int, int, int], float] = {}

    # Iterate over each trade record
    for _, row in trade_data.iterrows():
        i = int(row["origin_port_id"])
        j = int(row["dest_port_id"])
        k = int(row["k"])

        # Only compute revenue if we have a known distance
        dist_key = (i, j)
        if dist_key not in distances:
            continue

        distance_nm = distances[dist_key]

        # Only proceed if we have freight-rate information for this commodity
        if k not in rate_lookup:
            continue

        dist_ref, fr_low, fr_high = rate_lookup[k]

        # Determine which rate to use based on distance
        if distance_nm <= 1000:
            chosen_rate = fr_low
        elif distance_nm >= dist_ref:
            chosen_rate = fr_high
        else:
            # Linear interpolation between (1000 → fr_low) and (dist_ref → fr_high)
            fraction = (distance_nm - 1000) / (dist_ref - 1000)
            chosen_rate = fr_low + fraction * (fr_high - fr_low)

        # Scale by vessel capacity to get total revenue for that arc
        revenue[(i, j, k)] = chosen_rate * vessel_capacity

    return revenue

In [ ]:
def _get_active_arcs(xvars: dict, threshold: float = 0.5) -> dict:
    """
    Helper to extract arcs where variable value > threshold.

    Returns a dict: {vessel: [(i,j), ...]}.
    """
    active = defaultdict(list)
    for (v, i, j), var in xvars.items():
        try:
            if var.X > threshold:
                active[v].append((i, j))
        except AttributeError:
            # If callback context: var is not a Gurobi Var but callback access
            if xvars[v, i, j].Xn > threshold:
                active[v].append((i, j))
    return active


class SubtourEliminationCallback:
    """
    Gurobi callback for eliminating subtours in a vehicle routing context.

    Each vessel v has its own set of feasible arcs (i,j).
    At an integer solution, we identify connected components excluding the home (depot).
    If a component forms a subtour (disconnected from home), we add a lazy constraint
    to cut it off.
    """

    def __init__(self, xvars: dict, feasible_arcs: dict[int, set[tuple[int, int]]], home_port: int):
        """
        Args:
            xvars: dict-like mapping (v,i,j) -> Gurobi variable
            feasible_arcs: mapping vessel v -> set of allowed (i,j)
            home_port: index of the depot port
        """
        self.xvars = xvars
        self.feasible_arcs = feasible_arcs
        self.home_port = home_port

    def __call__(self, model: gp.Model, where: int):
        if where != gp.GRB.Callback.MIPSOL:
            return

        # Iterate over vessels to find used arcs
        for vessel, arcset in self.feasible_arcs.items():
            # Identify arcs with value > 0.5 in the current solution
            used_arcs = []
            for (i, j) in arcset:
                val = model.cbGetSolution(self.xvars[vessel, i, j])
                if val > 0.5:
                    used_arcs.append((i, j))

            if not used_arcs:
                continue  # no arcs used for this vessel

            # Build directed adjacency list
            adjacency = defaultdict(list)
            nodes = set()
            for i, j in used_arcs:
                adjacency[i].append(j)
                nodes.add(i)
                nodes.add(j)

            visited = set()
            # Examine each connected component via BFS/DFS
            for node in nodes:
                if node in visited:
                    continue
                queue = deque([node])
                component = []

                while queue:
                    current = queue.popleft()
                    if current in visited:
                        continue
                    visited.add(current)
                    component.append(current)
                    for neigh in adjacency.get(current, []):
                        if neigh not in visited:
                            queue.append(neigh)

                # If component does not include home and is smaller than full set, it's a subtour
                if self.home_port not in component and len(component) < len(nodes):
                    # All internal arcs within this component
                    cut_arcs = [(i, j) for (i, j) in arcset if i in component and j in component]
                    if cut_arcs:
                        expr = gp.quicksum(self.xvars[vessel, i, j] for (i, j) in cut_arcs)
                        # Add lazy constraint: sum(x in cut_arcs) <= |component| - 1
                        model.cbLazy(expr <= len(component) - 1)


def plot_routes_searoute(
    routes: dict[int, list[int]],
    coords: dict[int, tuple[float, float]],
    home_port: int | None = None,
):
    """
    Plot vessel routes on a global map using Plotly and Searoute.

    Args:
        routes: mapping vessel -> ordered list of port IDs visited sequentially
        coords: mapping port ID -> (lat, lon)
        home_port: optional port ID to highlight as depot
    """
    fig = go.Figure()
    # Predefined palette for up to 10 vessels
    base_colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'cyan', 'magenta', 'gray']

    for idx, (vessel, legs) in enumerate(routes.items()):
        color = base_colors[idx % len(base_colors)]
        # Draw each leg segment via Searoute
        for u, w in zip(legs[:-1], legs[1:]):
            lat1, lon1 = coords[u]
            lat2, lon2 = coords[w]
            try:
                route = sr.searoute((lat1, lon1), (lat2, lon2))
                lons, lats = zip(*route.geometry['coordinates'])
            except Exception:
                # Fallback to straight line if Searoute fails
                lons, lats = [lon1, lon2], [lat1, lat2]

            fig.add_trace(
                go.Scattergeo(
                    lat=lats,
                    lon=lons,
                    mode='lines',
                    line=dict(width=2, color=color),
                    name=f'Vessel {vessel}',
                    showlegend=False,
                )
            )
        # Mark ports on the route
        for port in legs:
            lat, lon = coords[port]
            fig.add_trace(
                go.Scattergeo(
                    lat=[lat],
                    lon=[lon],
                    mode='markers+text',
                    marker=dict(size=6, color=color),
                    text=str(port), textposition='top center',
                    showlegend=False,
                )
            )

    # Highlight home port if given
    if home_port is not None and home_port in coords:
        lat0, lon0 = coords[home_port]
        fig.add_trace(
            go.Scattergeo(
                lat=[lat0],
                lon=[lon0],
                mode='markers+text',
                marker=dict(size=12, color='black', symbol='star'),
                text='Home', textposition='bottom right',
                name='Home port',
            )
        )

    fig.update_layout(
        title='Optimised Fleet Routes',
        geo=dict(
            projection_type='natural earth',
            showland=True,
            landcolor='rgb(230,230,230)',
            countrycolor='gray',
            coastlinecolor='gray',
        ),
        margin=dict(l=0, r=0, t=40, b=0),
    )
    fig.show()


def plot_active_legs(
    xb_vars: dict[tuple[int, int, int], gp.Var],
    xl_vars: dict[tuple[int, int, int, int], gp.Var],
    coords: dict[int, tuple[float, float]],
    home_port: int | None = None,
    threshold: float = 0.5,
) -> list[go.Figure]:
    """
    Create a Plotly figure per vessel showing active loaded vs. ballast legs.

    Args:
        xb_vars: ballast-leg variables keyed by (vessel, from_port, to_port)
        xl_vars: loaded-leg variables keyed by (vessel, from_port, to_port, commodity)
        coords: mapping port ID -> (lat, lon)
        home_port: optional depot port to highlight
        threshold: cutoff for considering a var "active"

    Returns:
        List of Plotly Figure objects, one per vessel.
    """
    # Extract arcs where decision var > threshold
    ballast_arcs = defaultdict(list)
    for (v, i, j), var in xb_vars.items():
        if var.X > threshold:
            ballast_arcs[v].append((i, j))

    loaded_arcs = defaultdict(list)
    for (v, i, j, k), var in xl_vars.items():
        if var.X > threshold:
            loaded_arcs[v].append((i, j))

    vessels = set(ballast_arcs) | set(loaded_arcs)
    figures = []

    # Style definitions
    STYLES = {
        'loaded': {'color': 'blue', 'dash': 'solid', 'width': 3},
        'ballast': {'color': 'red', 'dash': 'dash', 'width': 2},
    }

    for vessel in vessels:
        fig = go.Figure()

        def add_arcs(arcs: list[tuple[int, int]], style: dict, label: str):
            """Plot each (i,j) as a line with given style and label."""
            first_trace = True
            for i, j in arcs:
                lat1, lon1 = coords[i]
                lat2, lon2 = coords[j]
                try:
                    route = sr.searoute((lat1, lon1), (lat2, lon2))
                    lons, lats = zip(*route.geometry['coordinates'])
                except Exception:
                    # Fall back to straight-line
                    lat1, lon1 = coords[i]
                    lat2, lon2 = coords[j]
                    lons, lats = [lon1, lon2], [lat1, lat2]

                fig.add_trace(
                    go.Scattergeo(
                        lat=lats,
                        lon=lons,
                        mode='lines',
                        line=dict(
                            color=style['color'],
                            dash=style['dash'],
                            width=style['width'],
                        ),
                        name=label if first_trace else None,
                        showlegend=first_trace,
                    )
                )
                first_trace = False

        # Plot loaded legs (solid blue)
        add_arcs(loaded_arcs.get(vessel, []), STYLES['loaded'], 'Loaded leg')
        # Plot ballast legs (dashed red)
        add_arcs(ballast_arcs.get(vessel, []), STYLES['ballast'], 'Ballast leg')

        # Plot visited ports (exclude home)
        visited = {
            p for (i, j) in ballast_arcs.get(vessel, []) + loaded_arcs.get(vessel, []) for p in (i, j)
        }
        if home_port is not None:
            visited.discard(home_port)

        for port in visited:
            lat, lon = coords[port]
            fig.add_trace(
                go.Scattergeo(
                    lat=[lat],
                    lon=[lon],
                    mode='markers+text',
                    marker=dict(size=6, color='black'),
                    text=str(port),
                    textposition='top center',
                    showlegend=False,
                )
            )

        # Highlight home port
        if home_port is not None and home_port in coords:
            lat0, lon0 = coords[home_port]
            fig.add_trace(
                go.Scattergeo(
                    lat=[lat0],
                    lon=[lon0],
                    mode='markers+text',
                    marker=dict(size=9, color='black', symbol='star'),
                    text='Home',
                    textposition='bottom center',
                    showlegend=False,
                )
            )

        fig.update_layout(
            title=f"Vessel {vessel} – Active Legs",
            geo=dict(
                projection_type='natural earth',
                showland=True,
                landcolor='rgb(230,230,230)',
                countrycolor='gray',
                coastlinecolor='gray',
            ),
            height=600,
            width=900,
            legend=dict(x=0.85, y=0.95),
        )
        figures.append(fig)

    return figures


def pretty_print_fleet_solution(
    model: gp.Model,
    vessel_list: list[int],
    vessel_catalog: dict[int, any],
    leg_data: dict[int, tuple[int, int, int, any, any]],
    xl_vars: dict[tuple[int, int, int, int], gp.Var],
    xb_vars: dict[tuple[int, int, int], gp.Var],
    cleaning_arc_vars: dict[tuple[int, int, int, int, int], gp.Var],
    cleaning_port_vars: dict[tuple[int, int, int, int], gp.Var],
    load_port_vars: dict[tuple[int, int, int], gp.Var],
    unload_port_vars: dict[tuple[int, int, int], gp.Var],
    serve_vars: dict[int, gp.Var],
    buy_vars: dict[int, gp.Var],
    fuel_cost_expr: gp.LinExpr,
    opex_cost_expr: gp.LinExpr,
    port_cost_expr: gp.LinExpr,
    handling_opex_expr: gp.LinExpr,
    cleaning_cost_expr: gp.LinExpr,
    capex_expr: gp.LinExpr,
    penalty_cost_expr: gp.LinExpr,
    total_cost: gp.LinExpr,
):
    """
    Print a readable summary of the fleet solution, including revenue, costs,
    leg assignments, and vessel-specific operations.

    Args:
        model: Gurobi model after optimization (with ObjVal)
        vessel_list: list of vessel IDs in the model
        vessel_catalog: metadata about vessels (e.g., capacities)
        leg_data: mapping leg index -> (origin, dest, commodity, ..., orig_leg_id)
        xl_vars: loaded-leg decision variables
        xb_vars: ballast-leg decision variables
        cleaning_arc_vars: at-sea cleaning decision variables
        cleaning_port_vars: at-port cleaning decision variables
        load_port_vars: loading decision variables per port and k
        unload_port_vars: unloading decision variables per port and k
        serve_vars: whether leg ℓ is served
        buy_vars: whether vessel v is purchased
        <cost_expr>: Gurobi expressions for each cost component
        total_cost: overall cost expression
    """
    print("🌍 GLOBAL MODEL SUMMARY")
    # Revenue is model objective + total_cost (if objective is profit)
    print(f"  Revenue:       ${model.ObjVal + total_cost.getValue():,.0f}")
    print(f"  Total Cost:    ${total_cost.getValue():,.0f}")
    print(f"  Profit (Obj):  ${model.ObjVal:,.0f}")
    print(f"  Penalties:     ${penalty_cost_expr.getValue():,.0f}\n")

    print("📦 LEG ASSIGNMENTS:")
    for leg_id, data in leg_data.items():
        origin, dest, commodity, _, orig_id = data
        status = "✅ Served" if serve_vars[leg_id].X > 0.5 else "❌ Not Served"
        print(f"  Leg {leg_id}: {origin} → {dest}, commodity {commodity} — {status}")

    print("\n🛳️ VESSEL OPERATIONS:")
    for v in vessel_list:
        if buy_vars[v].X < 0.5:
            continue  # vessel not used
        print(f"  Vessel {v} Operations:")

        # Ballast (empty) legs
        ballast_used = [
            (i, j)
            for (vv, i, j) in xb_vars
            if vv == v and xb_vars[vv, i, j].X > 0.5
        ]
        for i, j in ballast_used:
            print(f"    - Ballast: {i} → {j}")

        # Loaded (cargo) legs
        loaded_used = [
            (i, j, k)
            for (vv, i, j, k) in xl_vars
            if vv == v and xl_vars[vv, i, j, k].X > 0.5
        ]
        for i, j, k in loaded_used:
            print(f"    - Loaded: {i} → {j}, cargo {k}")

        # At-sea cleaning operations
        sea_cleans = [
            (i, j, k1, k2)
            for (vv, i, j, k1, k2) in cleaning_arc_vars
            if vv == v and cleaning_arc_vars[vv, i, j, k1, k2].X > 0.5
        ]
        for i, j, k1, k2 in sea_cleans:
            print(f"    - Cleaning at sea: {i} → {j} (switch {k1}→{k2})")

        # At-port cleaning operations
        port_cleans = [
            (p, k1, k2)
            for (vv, p, k1, k2) in cleaning_port_vars
            if vv == v and cleaning_port_vars[vv, p, k1, k2].X > 0.5
        ]
        for p, k1, k2 in port_cleans:
            print(f"    - Cleaning at port {p} (switch {k1}→{k2})")

        # Loading at ports
        load_ops = [
            (p, k)
            for (vv, p, k) in load_port_vars
            if vv == v and load_port_vars[vv, p, k].X > 0.5
        ]
        for p, k in load_ops:
            print(f"    - Loading at port {p}, commodity {k}")

        # Unloading at ports
        unload_ops = [
            (p, k)
            for (vv, p, k) in unload_port_vars
            if vv == v and unload_port_vars[vv, p, k].X > 0.5
        ]
        for p, k in unload_ops:
            print(f"    - Unloading at port {p}, commodity {k}")

    print("\n💰 COST BREAKDOWN:")
    print(f"  Fuel Cost:        ${fuel_cost_expr.getValue():,.0f}")
    print(f"  Opex Cost:        ${opex_cost_expr.getValue():,.0f}")
    print(f"  Port Fees:        ${port_cost_expr.getValue():,.0f}")
    print(f"  Handling Opex:    ${handling_opex_expr.getValue():,.0f}")
    print(f"  Cleaning Cost:    ${cleaning_cost_expr.getValue():,.0f}")
    print(f"  Capex (purchase): ${capex_expr.getValue():,.0f}")



In [ ]:

"""
Utility helpers for a **multi‑vessel VRP** with tankers, bulkers, and combination carriers.

* generate_random_demands(...) – build a yearly spot‑market demand set that is **compatible** with
  each port's export / import permissions.
* make_vessel_catalog(...)      – create a list/dict of candidate vessels together with purchase price
  and the technical data blocks that your existing per‑vessel route model needs.

Both helpers are pure‑Python and have **no external dependencies** beyond numpy / pandas.
"""
# ---------------------------------------------------------------------------
# 1)  Demand‑generator -------------------------------------------------------
# ---------------------------------------------------------------------------
def generate_random_demands(
    export_compat: Dict[int, set],
    import_compat: Dict[int, set],
    freight_rate_df: pd.DataFrame | None = None,
    rng: random.Random | None = None,
    n_legs: int = 100,
    qty_range: Tuple[int, int] = (20_000, 100_000),
) -> pd.DataFrame:
    """Return a *spot* demand table with at most ``n_legs`` profitable cargoes.

    Every leg (o,d,k) is feasible by construction:
        • ``k`` ∈ export_compat[o]
        • ``k`` ∈ import_compat[d]

    Parameters
    ----------
    export_compat / import_compat : ``{port_id: set[commodity]}``
    freight_rate_df              : optional df with index *k* and column ``rate``
    rng                          : seeded ``random.Random`` for reproducibility
    n_legs                       : upper bound on number of spot legs generated
    qty_range                    : uniform range for cargo quantity in **t** (metric tonnes)

    Returns
    -------
    pandas.DataFrame with columns::
        leg_id  origin  dest  k  qty_t  rate_usd_pt  rev_usd
    """
    rng = rng or random.Random()

    # ---- 1.1  candidate OD pairs -----------------------------------------
    feasible_triples: list[Tuple[int, int, int]] = []
    for o, export_k in export_compat.items():
        for d, import_k in import_compat.items():
            common_k = export_k & import_k
            feasible_triples.extend([(o, d, k) for k in common_k if o != d])

    rng.shuffle(feasible_triples)
    triples_use = feasible_triples[:n_legs]

    # ---- 1.2  assemble dataframe -----------------------------------------
    legs: list[dict] = []
    for leg_id, (o, d, k) in enumerate(triples_use):
        qty = 90000
        if freight_rate_df is not None and k in freight_rate_df.index:
            rate = float(freight_rate_df.loc[k, "rate"])
        else:
            # crude fallback – tweak as you wish
            rate = rng.uniform(8.0, 20.0)  # USD per tonne
        rev = rate * qty
        legs.append({
            "leg_id":  leg_id,
            "origin":  o,
            "dest":    d,
            "k":       k,
            "qty_t":   qty,
            "rate_usd_pt": rate,
            "rev_usd": rev,
        })

    return pd.DataFrame(legs)
# ---------------------------------------------------------------------------
# 2)  Vessel catalog & tech‑data wrapper ------------------------------------
# ---------------------------------------------------------------------------
@dataclass
class FuelCurve:
    speed_kn: float   # design speed (knots)
    fuel_tpd: float   # fuel consumption at design speed (t/day)

@dataclass
class VesselTechSpec:
    capacity_t: int
    ballast: FuelCurve
    laden: FuelCurve
    opex_usd_day: float

@dataclass
class Vessel:
    vessel_id: str
    vtype: str                     # "Tanker" | "Bulker" | "Combo"
    price_usd: float               # purchase (CAPEX) or 1y bareboat hire
    tech: VesselTechSpec

def make_vessel_catalog(
    base_specs: Dict[str, dict],
    n_each: Union[int, Dict[str, int]] = 3,
    prefix_map: Optional[Dict[str, str]] = None,
    seed: Optional[int] = 42,
) -> Dict[str, Vessel]:
    """
    """
    rng = random.Random(seed)
    prefix_map = prefix_map or {"Tanker": "T", "Bulker": "B", "Combi": "C"}

    # Determine how many of each type to create
    if isinstance(n_each, dict):
        counts: Dict[str, int] = n_each
    else:
        counts = {vtype: n_each for vtype in base_specs}

    catalog: Dict[str, Vessel] = {}
    for vtype, spec in base_specs.items():
        count = counts.get(vtype, 0)
        for i in range(count):
            pid = prefix_map.get(vtype, vtype[0].upper())
            vid = f"{pid}{i:02d}"
            tech = VesselTechSpec(
                capacity_t   = spec["capacity_t"],
                ballast      = FuelCurve(spec["speed_ball_kn"],  spec["fuel_ball_tpd"]),
                laden        = FuelCurve(spec["speed_laden_kn"], spec["fuel_laden_tpd"]),
                opex_usd_day = spec["opex_usd_day"],
            )
            catalog[vid] = Vessel(
                vessel_id = vid,
                vtype     = vtype,
                price_usd = spec["price_usd"],
                tech      = tech,
            )
    return catalog


base_specs = {
        "tanker": {"capacity_t": 90_000, "price_usd": 0, "opex_usd_day": 7300,
                    "speed_ball_kn": 14, "fuel_ball_tpd": 31,
                    "speed_laden_kn": 13, "fuel_laden_tpd": 33},
        "bulker": {"capacity_t":90_000, "price_usd": 0, "opex_usd_day": 6500,
                    "speed_ball_kn": 14, "fuel_ball_tpd": 31,
                    "speed_laden_kn": 13, "fuel_laden_tpd": 33},
        "combi":  {"capacity_t":90_000, "price_usd": 0, "opex_usd_day": 7900,
                    "speed_ball_kn": 14, "fuel_ball_tpd": 34,
                    "speed_laden_kn": 13, "fuel_laden_tpd": 36},
    }

In [ ]:
# ---------------------------------------------------------------------------
# 1) Load core data & define basic sets/constants
# ---------------------------------------------------------------------------

# Load the main export/import trade table
data = pd.read_excel('Final_export_import_data.xlsx')

# Load list of all port IDs from the distance matrix
N = pd.read_excel('distance_matrix.xlsx')['port_id'].to_list()  # Ports

# Define vessel types used in the model
V = ['combi', 'tanker', 'bulker']

# Fuel cost per tonne (USD/tonne)
fuel_cost = 550

# Fuel consumption rates by vessel type (t/day)
fuel_usage = pd.read_csv('Fuel_usage_vessels.csv')

# List of commodity codes (HS6) considered in the model
K = [
    100199, 250100, 251710, 252100, 252310, 252329,
    260111, 260112, 260400, 260600, 261800,
    270112, 270119, 270210, 270900, 271000
]

# ---------------------------------------------------------------------------
# 2) Commodity-vessel compliance rules
# ---------------------------------------------------------------------------

# For each vessel type, define the set of commodity codes it can carry
compliance_dict = {
    'combi': {
        100199, 250100, 251710, 252100, 252310, 252329,
        260111, 260112, 260400, 260600, 261800,
        270112, 270119, 270210, 270900, 271000
    },
    'tanker': {270900, 271000},  # Only crude/refined oils
    'bulker': {
        100199, 250100, 251710, 252100, 252310, 252329,
        260111, 260112, 260400, 260600, 261800,
        270112, 270119, 270210
    }
}

# ---------------------------------------------------------------------------
# 3) Build distance dictionary from distance matrix
# ---------------------------------------------------------------------------

# Read the full distance matrix (square), with port_id in first column
Distance_matrix = pd.read_excel('distance_matrix.xlsx')

# Convert column headers to strings (so they can be cast to int safely)
Distance_matrix.columns = Distance_matrix.columns.map(str)

# Flatten the distance matrix into a dict {(i, j): distance}
d: dict[tuple[int, int], float] = {}
for i_row in Distance_matrix.itertuples(index=False):
    i = int(i_row[0])  # origin port_id
    for j_str, dist in zip(Distance_matrix.columns[1:], i_row[1:]):
        if not pd.isna(dist):
            j = int(j_str)
            d[(i, j)] = float(dist)

# Load freight-rate model (contains HS6, Distance_nm, FR low, FR High)
freight_rate_df = pd.read_excel('freight_rate_model.xlsx')

# ---------------------------------------------------------------------------
# 4) Load export/import compatibility by port
# ---------------------------------------------------------------------------

# Read CSVs: each row has 'port_id' and a text list of commodity codes under column 'k'
K_export_port = pd.read_csv('exports_by_port.csv')
K_import_port = pd.read_csv('imports_by_port.csv')

# Helper to parse the 'k' column:
#   - strip brackets, flatten newlines, split on whitespace, convert to int
K_export_port['k'] = K_export_port['k'].apply(
    lambda x: list(map(int, x.strip('[]').replace('\n', ' ').split()))
)
K_import_port['k'] = K_import_port['k'].apply(
    lambda x: list(map(int, x.strip('[]').replace('\n', ' ').split()))
)

# Remove commodity code 151190 from all lists (not allowed)
K_export_port['k'] = K_export_port['k'].apply(lambda ks: [x for x in ks if x != 151190])
K_import_port['k'] = K_import_port['k'].apply(lambda ks: [x for x in ks if x != 151190])

# Build lookup dicts: port_id → set of compatible commodity codes
export_compat = {row.port_id: set(row.k) for row in K_export_port.itertuples()}
import_compat = {row.port_id: set(row.k) for row in K_import_port.itertuples()}

# ---------------------------------------------------------------------------
# 5) Load tank-cleaning time matrix
# ---------------------------------------------------------------------------

# Read the asymmetric cleaning-time matrix (rows/columns = HS6 codes)
cleaning_time_matrix = pd.read_excel("asymmetric_cleaning_time_matrix.xlsx")

# Convert cleaning_time_matrix into a dict {(k1, k2): time_in_days}
cleaning_time: dict[tuple[int, int], float] = {}
for i_code in cleaning_time_matrix['Unnamed: 0']:
    for j_code in cleaning_time_matrix.columns[1:]:
        # Look up the single value at row i_code, column j_code
        time_val = cleaning_time_matrix.loc[
            cleaning_time_matrix['Unnamed: 0'] == i_code, j_code
        ].values[0]
        cleaning_time[(int(i_code), int(j_code))] = float(time_val)

# Flat cleaning cost per operation at sea (USD)
cleaning_cost_pd = 10_000

# ---------------------------------------------------------------------------
# 6) Load/unload rates by commodity (tonnes per day)
# ---------------------------------------------------------------------------

# Loading rate (t/day) for each commodity code (dry-bulk all 22,572 tpd; liquids 63,216 tpd)
load_tpd: dict[int, int] = {
    100199: 22572, 250100: 22572, 251710: 22572, 252100: 22572,
    252310: 22572, 252329: 22572, 260111: 22572, 260112: 22572,
    260400: 22572, 260600: 22572, 261800: 22572, 270112: 22572,
    270119: 22572, 270210: 22572,  # all dry-bulk
    270900: 63216, 271000: 63216   # liquid crude & refined oils
}

# Unloading rate (t/day) for each commodity code
unload_tpd: dict[int, int] = {
    # Dry-bulk (grabs, conveyors, pneumatic unloaders)
    100199: 17000, 250100: 17000, 251710: 17000, 252100: 17000,
    252310: 17000, 252329: 17000, 260111: 17000, 260112: 17000,
    260400: 17000, 260600: 17000, 261800: 17000, 270112: 17000,
    270119: 17000, 270210: 17000,  # all solid bulk

    # Liquid cargoes discharged through pumps or cargo lines
    270900: 51840, 271000: 51840    # crude & refined oils
}

# ---------------------------------------------------------------------------
# 7) Build port-coordinate lookup from trade data
# ---------------------------------------------------------------------------

# coords: port_id → (latitude, longitude)
coords: dict[int, tuple[float, float]] = {}

# Populate coordinates for origin_port_id → (lat_x, lon_x)
for row in data.itertuples():
    coords[row.origin_port_id] = (row.lat_x, row.lon_x)

# Populate coordinates for dest_port_id → (lat_y, lon_y)
for row in data.itertuples():
    coords[row.dest_port_id] = (row.lat_y, row.lon_y)


### MODEL BUILDING AND MAIN

In [ ]:
```python
# ---------------------------------------------------------------------------
# main routine ---------------------------------------------------------------
# ---------------------------------------------------------------------------

def build_and_solve_fleet_model(
    home_port: int,
    demand_df: pd.DataFrame,                      # cols: origin, dest, k, rev_usd, leg_id
    vessel_catalog,                               # dict[str, Vessel]
    distances: Dict[Tuple[int, int], float],
    fuel_cost: float,
    O_compliance_dict: Dict[str, Set[int]],
    port_fee: float,
    load_tpd: Dict[int, float],
    unload_tpd: Dict[int, float],
    cleaning_time: Dict[Tuple[int, int], float],
    cleaning_cost_pd: float,
    unserved_penalty: float,
    *,
    max_stops: int = 15,
    max_voyage_days: float = 365.0,
    bigM: float | None = None,
    timelimit: int | None = None,
    mipgap: float = 0.1,
):
    """Return (model, routes_dict, assignments_df, port2cent, cent2ports)."""

    # ------------------------------------------------------------------
    # 0)  Sets & compressed graph --------------------------------------
    # ------------------------------------------------------------------

    # List of vessel IDs drawn from the catalog
    V = list(vessel_catalog)

    # Set of all demanded cargo types
    cargo_types = set(demand_df['k'])

    # Build a vessel-specific compliance dictionary:
    # only keep commodity codes that appear in demand_df
    compliance_dict = {
        v: {k for k in O_compliance_dict[v] if k in cargo_types}
        for v in V
    }

    # Construct list of demand legs (origin, dest, commodity, revenue, leg_id),
    # filtering out any trivial legs where origin == destination
    legs = []
    for row in demand_df.itertuples():
        o, d, k, rev, lid = row.origin, row.dest, row.k, row.rev_usd, row.leg_id
        if o != d:
            legs.append((o, d, k, rev, lid))

    # Index range for legs
    Leg = range(len(legs))
    # Map each index ℓ to its data tuple
    leg_data = {ℓ: legs[ℓ] for ℓ in Leg}

    # Set of all ports involved, including home_port
    all_ports = {home_port, *demand_df["origin"], *demand_df["dest"]}

    # Pre-filter available arcs (i,j) from distances:
    # only if i != j and both ports are in all_ports
    arcs_all = {
        (i, j): d_ij
        for (i, j), d_ij in distances.items()
        if i != j and i in all_ports and j in all_ports
    }

    # Initialize per-vessel data structures
    P_use       = {v: {home_port} for v in V}         # Ports vessel v might visit
    arcsCARGO   = {v: set() for v in V}               # Cargo arcs (i,j) per vessel
    arcsBALLAST = {v: set() for v in V}               # Ballast arcs (i,j) per vessel
    arc2cargos  = {v: defaultdict(set) for v in V}    # Map (i,j)→set of cargo types possible on that arc
    inbound     = {v: defaultdict(set) for v in V}    # For each port p, which cargo types arrive there
    outbound    = {v: defaultdict(set) for v in V}    # For each port p, which cargo types depart from there
    cb_index    = []  # List of (v, i, j, k1, k2) for at-sea cleaning switches
    cP_index    = []  # List of (v, p, k1, k2) for in-port cleaning switches

    # Build graph components per vessel
    for v in V:
        Kv = compliance_dict[v]  # Allowed commodity codes for vessel v

        # Identify cargo arcs: if (o,d) is demanded, k ∈ Kv, and arc exists in arcs_all
        for o, d, k, rev, lid in legs:
            if k in Kv and (o, d) in arcs_all:
                arcsCARGO[v].add((o, d))
                arc2cargos[v][(o, d)].add(k)
                P_use[v].update([o, d])
                outbound[v][o].add(k)
                inbound[v][d].add(k)

        # Ballast arcs: connect any two reachable ports (including home_port)
        reachable = {p for arc in arcsCARGO[v] for p in arc} | {home_port}
        for i in reachable:
            for j in reachable:
                if i != j and (i, j) in arcs_all:
                    arcsBALLAST[v].add((i, j))

        # Build the cb_index list: at-sea cleaning switches where
        # vessel arrives with k1 and departs with k2 (k1 ≠ k2)
        for (i, j) in arcsBALLAST[v]:
            for k1 in inbound[v][i]:
                for k2 in outbound[v][j]:
                    if k1 in Kv and k2 in Kv and k1 != k2:
                        cb_index.append((v, i, j, k1, k2))

        # Build the cP_index list: possible in-port cleaning where
        # vessel switches from k1 (arrived) to k2 (departing)
        for p in P_use[v]:
            incoming = inbound[v][p]
            outgoing = outbound[v][p]
            for k2 in outgoing:
                if k2 not in Kv:
                    continue
                for k1 in Kv:
                    if k1 != k2:
                        cP_index.append((v, p, k1, k2))

    # Final arc set per vessel: union of cargo and ballast arcs
    dist_use = {v: arcsCARGO[v] | arcsBALLAST[v] for v in V}


    # ------------------------------------------------------------------
    # 1)  Instantiate variables & preprocess tech parameters -----------
    # ------------------------------------------------------------------

    # Speeds in nautical miles per day (nm/day) for ballast and laden
    speed_ball = {v: vessel_catalog[v].tech.ballast.speed_kn * 24 for v in V}
    speed_lad  = {v: vessel_catalog[v].tech.laden.speed_kn * 24 for v in V}

    # Fuel consumption (t/day) for ballast and laden
    fuel_b_tpd = {v: vessel_catalog[v].tech.ballast.fuel_tpd for v in V}
    fuel_l_tpd = {v: vessel_catalog[v].tech.laden.fuel_tpd for v in V}


    # ------------------------------------------------------------------
    # 2)  Build Gurobi model -------------------------------------------
    # ------------------------------------------------------------------

    # Create Gurobi environment and model
    env = gp.Env(params=options)
    m = gp.Model("FleetOptimVRP_Full", env=env)

    # Set solver parameters
    if timelimit:
        m.setParam("TimeLimit", timelimit)
    m.setParam("MIPGap", mipgap)

    # ---------------- Decision variables ------------------------------

    # Binary: whether to buy (or hire) vessel v
    buy = m.addVars(V, vtype=gp.GRB.BINARY, name="buy")

    # Binary arc usage decision x[v,i,j] for both cargo and ballast arcs
    x = m.addVars(
        [(v, i, j) for v in V for (i, j) in dist_use[v]],
        vtype=gp.GRB.BINARY, name="x"
    )

    # Binary ballast‐only arc usage xb[v,i,j]
    xb = m.addVars(
        [(v, i, j) for v in V for (i, j) in arcsBALLAST[v]],
        vtype=gp.GRB.BINARY, name="xb"
    )

    # Binary loaded‐cargo arc usage xl[v,i,j,k]
    xl = m.addVars(
        [(v, i, j, k)
         for v in V
         for (i, j) in arcsCARGO[v]
         for k in arc2cargos[v][(i, j)]
        ],
        vtype=gp.GRB.BINARY, name="xl"
    )

    # Binary at-sea cleaning decision cb[v,i,j,k1,k2]
    cb = m.addVars(cb_index, vtype=gp.GRB.BINARY, name="cb")

    # Binary in-port cleaning decision cP[v,p,k1,k2]
    cP = m.addVars(cP_index, vtype=gp.GRB.BINARY, name="cP")

    # Continuous: time spent at sea cleaning for each cb index
    tSea = m.addVars(cb_index, vtype=gp.GRB.CONTINUOUS, lb=0, name="tSea")

    # Binary triggers for loading/unloading at port: yL[v,p,k], yU[v,p,k]
    yL = m.addVars(
        [(v, p, k) for v in V for p in P_use[v] for k in compliance_dict[v]],
        vtype=gp.GRB.BINARY, name="yL"
    )
    yU = m.addVars(
        [(v, p, k) for v in V for p in P_use[v] for k in compliance_dict[v]],
        vtype=gp.GRB.BINARY, name="yU"
    )

    # Binary: whether leg ℓ is served
    serve = m.addVars(Leg, vtype=gp.GRB.BINARY, name="serve")

    # Continuous residual port‐cleaning time: resP[v,p,k1,k2]
    resP = m.addVars(cP.keys(), lb=0, name="residual_port")


    # ---------------- Objective ---------------------------------------

    # Revenue from served legs
    rev_expr = gp.quicksum(leg_data[ℓ][3] * serve[ℓ] for ℓ in Leg)

    # Prepare placeholders for cost expressions
    cii_num = {}      # CII numerator per vessel (CO2 emissions)
    cii_den = {}      # CII denominator per vessel (dwt * distance)
    eeoi_num = {}     # EEOI numerator per vessel (CO2 emissions)
    eeoi_den = {}     # EEOI denominator per vessel (cargo ton‐nm)
    CO2_factor = 3.114  # kg CO2 per tonne of fuel

    vessel_days = {}  # Total days vessel v is at sea + in port

    # Initialize cost accumulators
    fuel_cost_expr       = 0
    opex_cost_expr       = 0
    port_cost_expr       = 0
    handling_opex_expr   = 0
    cleaning_cost_expr   = 0
    all_visits           = {}

    # Build sub-expressions per vessel
    for v in V:
        Kv = compliance_dict[v]
        dwt = vessel_catalog[v].tech.capacity_t

        total_days = 0
        co2_expr = 0
        distance_expr = 0
        cargotonmile_expr = 0

        # Count how many times vessel v visits each port p
        visit_v = {
            p: gp.quicksum(x[v, i, j] for (i, j) in dist_use[v] if j == p)
            for p in P_use[v]
        }
        # Port fees = port_fee * number_of_visits
        port_cost_expr += port_fee * gp.quicksum(visit_v[p] for p in P_use[v])
        all_visits[v] = visit_v

        # Port operations: loading, unloading, plus in-port cleaning time
        for p in P_use[v]:
            port_ops_expr = (
                gp.quicksum(dwt / load_tpd[k] * yL[v, p, k] for k in Kv) +
                gp.quicksum(dwt / unload_tpd[k] * yU[v, p, k] for k in Kv) +
                gp.quicksum(
                    resP[v, p, k1, k2]
                    for (vv, pp, k1, k2) in cP_index
                    if vv == v and pp == p
                )
            )
            total_days += port_ops_expr
            handles_cost = vessel_catalog[v].tech.opex_usd_day * port_ops_expr
            handling_opex_expr += handles_cost

        # At-sea ballast legs: compute days, fuel, OPEX, CO2, distance
        for (i, j) in arcsBALLAST[v]:
            d_nm = distances[i, j]
            days_b = d_nm / speed_ball[v]
            fuel_b = fuel_b_tpd[v] * days_b

            total_days     += days_b * xb[v, i, j]
            fuel_cost_expr += fuel_b * fuel_cost * xb[v, i, j]
            opex_cost_expr += vessel_catalog[v].tech.opex_usd_day * days_b * xb[v, i, j]
            co2_expr       += fuel_b * CO2_factor * xb[v, i, j]
            distance_expr  += d_nm * xb[v, i, j]

        # At-sea laden (cargo) legs: compute days, fuel, OPEX, CO2, cargo ton‐nm
        for (i, j) in arcsCARGO[v]:
            d_nm = distances[i, j]
            days_l = d_nm / speed_lad[v]
            fuel_l = fuel_l_tpd[v] * days_l

            for k in arc2cargos[v][(i, j)]:
                total_days         += days_l * xl[v, i, j, k]
                fuel_cost_expr     += fuel_l * fuel_cost * xl[v, i, j, k]
                opex_cost_expr     += vessel_catalog[v].tech.opex_usd_day * days_l * xl[v, i, j, k]
                co2_expr           += fuel_l * CO2_factor * xl[v, i, j, k]
                cargotonmile_expr  += vessel_catalog[v].tech.capacity_t * d_nm * xl[v, i, j, k]
                distance_expr      += d_nm * xl[v, i, j, k]

        vessel_days[v] = total_days

        # Compute CII numerator/denominator if dwt > 0
        if dwt > 0:
            cii_num[v] = co2_expr
            cii_den[v] = dwt * distance_expr
        else:
            cii_num[v] = 0
            cii_den[v] = 0

        eeoi_num[v] = co2_expr
        eeoi_den[v] = cargotonmile_expr

    # Sea cleaning cost (at-sea time tSea)
    sea_clean_cost = gp.quicksum(
        cleaning_cost_pd * tSea[key]
        for key in tSea.keys()
    )

    # Port cleaning cost (residual port time resP)
    port_clean_cost = gp.quicksum(
        cleaning_cost_pd * resP[key]
        for key in resP.keys()
    )

    cleaning_cost_expr = sea_clean_cost + port_clean_cost

    # CapEx cost: purchase/hire cost times buy[v]
    capex_expr = gp.quicksum(vessel_catalog[v].price_usd * buy[v] for v in V)

    # Penalty for unserved legs
    penalty_cost_expr = gp.quicksum((1 - serve[ℓ]) * unserved_penalty for ℓ in Leg)

    # Total cost = sum of all cost components
    total_cost = (
        fuel_cost_expr
        + opex_cost_expr
        + port_cost_expr
        + handling_opex_expr
        + cleaning_cost_expr
        + capex_expr
        + penalty_cost_expr
    )

    # Maximize profit = revenue - total cost
    m.setObjective(rev_expr - total_cost, gp.GRB.MAXIMIZE)


    # ------------------------------------------------------------------
    # 2)  Constraints ---------------------------------------------------
    # ------------------------------------------------------------------

    # 2.1 Link x[v,i,j] to ballast vs. laden usage
    for v in V:
        for (i, j) in dist_use[v]:
            m.addConstr(
                x[v, i, j]
                == xb[v, i, j]
                + gp.quicksum(xl[v, i, j, k] for k in arc2cargos[v].get((i, j), [])),
                name=f"link_x_{v}_{i}_{j}"
            )

    # 2.2 At most one of ballast or loaded usage on each cargo‐capable arc
    for v in V:
        for (i, j), Ks in arc2cargos[v].items():
            if (i, j) in arcsBALLAST[v]:
                m.addConstr(
                    xb[v, i, j]
                    + gp.quicksum(xl[v, i, j, k] for k in Ks)
                    <= 1,
                    name=f"either_ballast_or_laden_{v}_{i}_{j}"
                )

    # 2.3 Purchase triggers: can only load/unload if vessel is bought
    m.addConstrs(
        (yL[v, p, k] <= buy[v] for v in V for p in P_use[v] for k in compliance_dict[v])
    )
    m.addConstrs(
        (yU[v, p, k] <= buy[v] for v in V for p in P_use[v] for k in compliance_dict[v])
    )

    # 2.4 One cargo type per arc per vessel
    m.addConstrs(
        (
            gp.quicksum(xl[v, i, j, k] for k in arc2cargos[v][(i, j)]) <= 1
            for v in V for (i, j) in arcsCARGO[v]
        ),
        name="one_cargo_per_arc"
    )

    # 2.5 Flow constraints per vessel
    for v in V:
        # 2.5.1 Departure from home_port: must use exactly one arc out if bought
        m.addConstr(
            gp.quicksum(
                x[v, home_port, j]
                for j in P_use[v]
                if (home_port, j) in dist_use[v]
            )
            == buy[v],
            name=f"Exit_home_{v}"
        )

        # 2.5.2 Return to home_port: must use exactly one arc in if bought
        m.addConstr(
            gp.quicksum(
                x[v, i, home_port]
                for i in P_use[v]
                if (i, home_port) in dist_use[v]
            )
            == buy[v],
            name=f"Enter_home_{v}"
        )

        # 2.5.3 Flow balance & no consecutive ballast arcs at intermediate ports
        for p in P_use[v]:
            if p == home_port:
                continue

            # Flow balance: inflow == outflow
            inflow = gp.quicksum(
                x[v, i, p]
                for i in P_use[v]
                if (i, p) in dist_use[v]
            )
            outflow = gp.quicksum(
                x[v, p, j]
                for j in P_use[v]
                if (p, j) in dist_use[v]
            )
            m.addConstr(inflow == outflow, name=f"flow_balance_{v}_{p}")

            # No two ballast arcs in a row: sum of in_b + out_b <= 1
            in_b = gp.quicksum(
                xb[v, i, p]
                for i in P_use[v]
                if (i, p) in arcsBALLAST[v]
            )
            out_b = gp.quicksum(
                xb[v, p, j]
                for j in P_use[v]
                if (p, j) in arcsBALLAST[v]
            )
            m.addConstr(in_b + out_b <= 1, name=f"no_two_ballast_{v}_{p}")

    # 2.6 Maximum stops per vessel
    m.addConstrs(
        (
            gp.quicksum(x[v, i, j] for (i, j) in dist_use[v])
            <= max_stops * buy[v]
            for v in V
        ),
        name="max_stops"
    )

    # 2.7 Demand coverage: if a leg ℓ can be served by any vessel, sum of xl == serve[ℓ]
    for ℓ in Leg:
        o, d, k, *_ = leg_data[ℓ]
        keys = [
            (v, o, d, k)
            for v in V
            if k in compliance_dict[v] and (v, o, d, k) in xl
        ]
        if not keys:
            continue
        m.addConstr(
            gp.quicksum(xl[key] for key in keys) == serve[ℓ],
            name=f"cov_{ℓ}"
        )

    # 2.8 Load/unload triggers: link arc usage to yL/yU
    for v in V:
        Kv = compliance_dict[v]
        for p in P_use[v]:
            for k in Kv:
                out_lad = gp.quicksum(
                    xl[v, p, j, k]
                    for j in P_use[v]
                    if (p, j) in arcsCARGO[v] and k in arc2cargos[v].get((p, j), set())
                )
                in_lad = gp.quicksum(
                    xl[v, i, p, k]
                    for i in P_use[v]
                    if (i, p) in arcsCARGO[v] and k in arc2cargos[v].get((i, p), set())
                )
                m.addConstr(out_lad == yL[v, p, k], name=f"out_lad_{v}_{p}_{k}")
                m.addConstr(in_lad == yU[v, p, k], name=f"in_lad_{v}_{p}_{k}")

    # 2.9 At-sea cleaning constraints (cb and tSea)
    for (v, i, j, k1, k2) in cb_index:
        if k1 == k2:
            continue

        # Vessel must be bought to do any cleaning
        m.addConstr(cb[v, i, j, k1, k2] <= buy[v])

        # Total at-sea cleaning time ≤ available ballast transit days
        days_b = distances[i, j] / speed_ball[v]
        full_req = cleaning_time[k1, k2]
        m.addConstr(
            tSea[v, i, j, k1, k2] <= days_b * cb[v, i, j, k1, k2],
            name=f"tSea_{v}_{i}_{j}_{k1}_{k2}"
        )
        m.addConstr(
            tSea[v, i, j, k1, k2] <= cleaning_time[k1, k2]
        )
        m.addConstr(
            tSea[v, i, j, k1, k2]
            + full_req * cP[v, j, k1, k2]
            >= full_req * cb[v, i, j, k1, k2],
            name=f"tSea_{v}_{i}_{j}_{k1}_{k2}"
        )

        # Cleaning switch logic: can only clean at sea if unloading and loading events occur
        m.addConstr(cb[v, i, j, k1, k2] <= yU[v, i, k1])
        m.addConstr(cb[v, i, j, k1, k2] <= xb[v, i, j])
        m.addConstr(cb[v, i, j, k1, k2] <= yL[v, j, k2])
        m.addConstr(
            cb[v, i, j, k1, k2]
            >= yU[v, i, k1] + xb[v, i, j] + yL[v, j, k2] - 2
        )

    # 2.10 In-port cleaning constraints (cP and resP)
    for (v, p, k1, k2) in cP_index:
        if k1 == k2:
            continue

        # Skip if unloading/loading variables don't exist
        if (v, p, k1) not in yU or (v, p, k2) not in yL:
            continue

        # If vessel switches from k1 to k2 at port p (yU + yL - 1), must trigger either in-port or at-sea cleaning
        switch_at_p = yU[v, p, k1] + yL[v, p, k2] - 1
        port_in_cb_sum = gp.quicksum(
            cb[v, i, p, k1, k2]
            for i in P_use[v]
            if (i, p) in arcsBALLAST[v] and (v, i, p, k1, k2) in cb
        )
        m.addConstr(
            cP[v, p, k1, k2] + port_in_cb_sum >= switch_at_p,
            name=f"port_cleaning_{v}_{p}_{k1}_{k2}"
        )

    # Residual port cleaning time bounds
    for (v, p, k1, k2) in cP_index:
        sea_total = gp.quicksum(
            tSea[v, i, p, k1, k2]
            for i in P_use[v]
            if (i, p) in arcsBALLAST[v] and (v, i, p, k1, k2) in tSea
        )

        m.addConstr(
            resP[v, p, k1, k2]
            >= cleaning_time[k1, k2] * cP[v, p, k1, k2]
            - sea_total,
            name=f"resP_lb_{v}_{p}_{k1}_{k2}"
        )
        m.addConstr(
            resP[v, p, k1, k2]
            <= cleaning_time[k1, k2] * cP[v, p, k1, k2],
            name=f"resP_ub_{v}_{p}_{k1}_{k2}"
        )

    # 2.11 Enforce maximum voyage days per vessel
    for v in V:
        if max_voyage_days > 0:
            m.addConstr(
                vessel_days[v] <= max_voyage_days,
                name=f"max_voyage_days_{v}"
            )


    # ------------------------------------------------------------------
    # 3)  Solve ---------------------------------------------------------
    # ------------------------------------------------------------------

    # Instruct Gurobi to write node files once memory usage reaches 50%
    m.setParam('NodefileStart', 0.5)
    # Specify directory for node files
    m.setParam('NodefileDir', '/tmp/gurobi_nodefiles')
    # Enable lazy constraints (for subtour elimination callback)
    m.Params.LazyConstraints = 1
    m.params.LogFile = 'CASE15legs.log'

    # Optimize with the subtour-elimination callback
    m.optimize(SubtourEliminationCallback2(x, dist_use, home_port))


    # ------------------------------------------------------------------
    # 4)  Extract solution ---------------------------------------------
    # ------------------------------------------------------------------

    routes = {}
    assign_rows = []

    # If the model is infeasible, compute IIS and print infeasible constraints
    if m.status == gp.GRB.INFEASIBLE:
        print("The model is infeasible. Computing IIS...")
        m.computeIIS()
        print("\nThe following constraints are infeasible:")
        for c in m.getConstrs():
            if c.IISConstr:
                print(f"Constraint: {c.ConstrName}")

    # If a feasible solution exists, extract relevant decisions
    if m.SolCount:
        for v in V:
            if buy[v].X < 0.5:
                continue
            # ... (additional route extraction logic would go here) ...
            pass

    # Return the model and all relevant data structures
    return (
        m, V, vessel_catalog, leg_data,
        x, xl, xb, cb, cP, yL, yU, serve, buy,
        fuel_cost_expr, opex_cost_expr, port_cost_expr,
        handling_opex_expr, cleaning_cost_expr,
        capex_expr, penalty_cost_expr, total_cost,
        cii_num, cii_den, eeoi_num, eeoi_den,
        vessel_days
    )
```


In [ ]:

def vessel_summary_df(
    V, buy, x,xl, distances, vessel_days,
    cii_num, cii_den, eeoi_num, eeoi_den,
     legs, threshold: float = 0.5,
):
    if isinstance(distances, dict):
        get_dist = distances.__getitem__
    else:
        def get_dist(key):
            i, j = key
            return distances.loc[i, j]
    arc2legs = (
    legs
      .groupby(["origin","dest"])["leg_id"]
      .apply(list)
      .to_dict()
)
    rows = []
    for v in V:
        if buy[v].X < threshold:
            continue
        dist_nm = sum(
            get_dist((i, j)) * var.X
            for (vv, i, j), var in x.items()
            if vv == v and var.X > threshold
        )
        days = vessel_days[v].getValue()
        CII  = cii_num[v].getValue()  / cii_den[v].getValue() * 1e6
        EEOI = eeoi_num[v].getValue() / eeoi_den[v].getValue() * 1e6

        legs_v = [
            leg_id
            for ((vv, i, j,k), var) in xl.items()
            if vv == v and var.X > threshold
            for leg_id in arc2legs.get((i, j), [])
        ]

        # sum up their revenue
        revenue = float(legs.set_index("leg_id")["rev_usd"].loc[legs_v].sum())

        rows.append(dict(
            vessel          = v,
            distance_nm     = dist_nm,
            days            = days,
            CII             = CII,
            EEOI            = EEOI,
            revenue_usd     = revenue,
        ))

    return (pd.DataFrame(rows)
            .set_index("vessel")
            .sort_index())


In [ ]:

# ---------------------------------------------------------------------------
# 1.  Define one‐scenario wrapper (build model → solve → gather outputs)
# ---------------------------------------------------------------------------
def run_scenario(
    demand_df: pd.DataFrame,
    scenario_id: str,
    compliance: dict,
    *,
    home_port: int = 0,
    fuel_price: float = 1000,
    timelimit: int = 5400,
    mipgap: float = 0,
):
    """
    Build, solve, and post‐process the fleet optimization for a single scenario.

    Args:
        demand_df       : DataFrame with columns [origin, dest, k, rev_usd, leg_id].
        scenario_id     : Unique identifier for this scenario (str).
        compliance      : Dict mapping vessel_id → set of allowable commodity codes.
    Keyword Args:
        home_port       : Port index used as home/depot (default: 0).
        fuel_price      : Fuel cost ($/tonne) for this scenario (default: 1000).
        timelimit       : Gurobi time limit in seconds (default: 5400).
        mipgap          : Gurobi relative MIP gap (default: 0).

    Returns:
        scenario_kpi (dict): Scenario‐level KPIs (objective, # vessels, legs, etc.).
        fleet_df      (DataFrame): Vessel‐level summary, with one row per vessel.
    """

    # ---- Build & solve the optimization model --------------------------------
    # The build_and_solve_fleet_model function returns the Gurobi model object 'm'
    # plus all intermediate data structures needed for downstream KPI extraction.
    (
        m,
        V,
        vessel_catalog,
        leg_data,
        x,
        xl,
        xb,
        cb,
        cP,
        yL,
        yU,
        serve,
        buy,
        fuel_cost_expr,
        opex_cost_expr,
        port_cost_expr,
        handling_opex_expr,
        cleaning_cost_expr,
        capex_expr,
        penalty_cost_expr,
        total_cost,
        cii_num,
        cii_den,
        eeoi_num,
        eeoi_den,
        vessel_days
    ) = build_and_solve_fleet_model(
        home_port         = home_port,
        demand_df         = demand_df,
        vessel_catalog    = fleet,          # global catalog object defined elsewhere
        distances         = d,              # prebuilt distance dict
        fuel_cost         = fuel_price,
        O_compliance_dict = compliance,
        port_fee          = 10_000,
        load_tpd          = load_tpd,
        unload_tpd        = unload_tpd,
        cleaning_time     = cleaning_time,
        cleaning_cost_pd  = 10_000,
        unserved_penalty  = 0,
        max_stops         = 100,
        max_voyage_days   = 365,
        timelimit         = timelimit,
        mipgap            = mipgap,
    )

    # ---- Scenario‐level KPIs --------------------------------------------------
    # Summarize key performance indicators at the scenario level:
    #   • scenario ID, number of legs, objective value,
    #   • number of vessels bought, number of ballast legs used, number of loaded legs used
    scenario_kpi = dict(
        scenario        = scenario_id,
        n_legs          = len(demand_df),
        objective       = m.objVal,
        vessels_bought  = sum(buy[v].X > 0.5 for v in V),
        ballast_legs    = sum(var.X > 0.5 for var in xb.values()),
        loaded_legs     = sum(var.X > 0.5 for var in xl.values()),
    )

    # ---- Vessel‐level KPIs ----------------------------------------------------
    # Build a DataFrame summarizing each vessel's performance:
    #   • use vessel_summary_df to compute relevant metrics (e.g., CII, EEOI, vessel days)
    fleet_df = vessel_summary_df(
        V,
        buy,
        x,
        xl,
        d,
        vessel_days,
        cii_num,
        cii_den,
        eeoi_num,
        eeoi_den,
        legs
    )

    # ---- Plot active legs on a map ------------------------------------------------
    # Use Plotly + sea‐route to visualize each vessel's active ballast & loaded legs
    figures = plot_active_legs_plotly2(
        xb.items(),
        xl.items(),
        coords,
        home_port = home_port
    )
    for fig in figures:
        fig.show()

    # ---- Pretty‐print the full fleet solution to console --------------------------------
    pretty_print_fleet_solution(
        m,
        V,
        vessel_catalog,
        leg_data,
        xl,
        xb,
        cb,
        cP,
        yL,
        yU,
        serve,
        buy,
        fuel_cost_expr,
        opex_cost_expr,
        port_cost_expr,
        handling_opex_expr,
        cleaning_cost_expr,
        capex_expr,
        penalty_cost_expr,
        total_cost
    )

    # Tag each row in fleet_df with the scenario ID for traceability
    fleet_df["scenario"] = scenario_id

    # ---- Clean up Gurobi objects to free memory ------------------------------------
    m.dispose()
    del m, x, xl, xb, vessel_days, cii_num, cii_den, eeoi_num, eeoi_den

    # Force garbage collection for large intermediate objects
    import gc
    gc.collect()

    # Return the scenario KPI dictionary and the vessel summary DataFrame
    return scenario_kpi, fleet_df



In [ ]:

# ---------------------------------------------------------------------------
# 2.  run many scenarios
# ---------------------------------------------------------------------------
from itertools import product
import random, pandas as pd

SCENARIOSCase = [
    # label, n_legs, random_seed
    ("10",  10,  14),
    ("20",  20,  15),
    ("30",  30,  25),
    ("40",  40,  36),
    ("50",  50,  40),
]
scenario_kpi_rows = []
fleet_frames      = []

fleet =make_vessel_catalog(
    base_specs,
    n_each={"combi": 2, "bulker": 0, "tanker":0})
print("\nCandidate fleet IDs:", list(fleet))

vessel_capacity = {v: fleet[v].tech.capacity_t for v in fleet}
compliance_dict2 = {
    vid: compliance_dict[fleet[vid].vtype.lower()]
    for vid in fleet
}

for label, n_legs, seed in SCENARIOSCase:
    rng  = random.Random(seed)
    legs = generate_random_demands(export_compat, import_compat,
                                   n_legs=n_legs, rng=rng)

    kpi_row, fleet_df = run_scenario(legs, label,compliance_dict2)

    scenario_kpi_rows.append(kpi_row)
    fleet_frames.append(fleet_df)

# ---------------------------------------------------------------------------
# 3.  gather everything in two tidy tables
# ---------------------------------------------------------------------------
scenario_kpi_df_1 = pd.DataFrame(scenario_kpi_rows).set_index("scenario")
fleet_grid_df_1   = pd.concat(fleet_frames).reset_index()   # keep vessel col


Candidate fleet IDs: ['C00', 'C01']
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2660920
Academic license 2660920 - for non-commercial use only - registered to an___@stud.ntnu.no
Set parameter TimeLimit to value 5400
Set parameter MIPGap to value 0
Set parameter NodefileStart to value 0.5
Set parameter NodefileDir to value "/tmp/gurobi_nodefiles"
Set parameter LazyConstraints to value 1
Set parameter LogFile to value "CASE15legs.log"
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  5400
MIPGap  0
NodefileStart  0.5
LazyConstraints  1

Academic license 2660920 - for non-commercial use only - registered to an___@stud.ntnu.no
Optimize a model with 3670 rows, 2452 columns and 11808 nonzeros
Model fingerprint: 0xd1550b94
Variable types

🌍 GLOBAL MODEL SUMMARY
  Revenue:       $12,091,915
  Total Cost:    $7,166,384
  Profit (Obj):  $4,925,532
  Penalties:     $0

📦 LEG ASSIGNMENTS:
  Leg 0: 13 → 3 cargo 252329 — ❌ Not Served
  Leg 1: 31 → 5 cargo 260111 — ✅ Served
  Leg 2: 35 → 8 cargo 260112 — ✅ Served
  Leg 3: 37 → 3 cargo 252310 — ✅ Served
  Leg 4: 4 → 22 cargo 260111 — ✅ Served
  Leg 5: 23 → 28 cargo 271000 — ✅ Served
  Leg 6: 17 → 26 cargo 270119 — ✅ Served
  Leg 7: 13 → 38 cargo 251710 — ✅ Served
  Leg 8: 14 → 29 cargo 270112 — ✅ Served
  Leg 9: 36 → 4 cargo 260112 — ✅ Served

🛳️ VESSEL OPERATIONS:
  🔄 Operations:
    - Sailed: 26 → 23 (Ballast: 1)
    - Sailed: 3 → 35 (Ballast: 1)
    - Sailed: 22 → 14 (Ballast: 1)
    - Sailed: 28 → 0 (Ballast: 1)
    - Sailed: 8 → 13 (Ballast: 1)
    - Sailed: 29 → 37 (Ballast: 1)
    - Sailed: 5 → 36 (Ballast: 1)
    - Sailed: 0 → 31 (Ballast: 1)
    - Sailed: 38 → 17 (Ballast: 1)
    - Cargo 260111: 31 → 5
    - Cargo 260112: 35 → 8
    - Cargo 270119: 17 → 26
    - Cargo 2

🌍 GLOBAL MODEL SUMMARY
  Revenue:       $19,915,339
  Total Cost:    $10,479,215
  Profit (Obj):  $9,436,124
  Penalties:     $0

📦 LEG ASSIGNMENTS:
  Leg 0: 5 → 27 cargo 271000 — ✅ Served
  Leg 1: 12 → 13 cargo 271000 — ✅ Served
  Leg 2: 9 → 42 cargo 271000 — ❌ Not Served
  Leg 3: 17 → 8 cargo 260112 — ❌ Not Served
  Leg 4: 5 → 0 cargo 271000 — ✅ Served
  Leg 5: 25 → 11 cargo 270900 — ✅ Served
  Leg 6: 3 → 28 cargo 271000 — ✅ Served
  Leg 7: 26 → 8 cargo 260400 — ✅ Served
  Leg 8: 28 → 31 cargo 270900 — ✅ Served
  Leg 9: 7 → 31 cargo 270900 — ✅ Served
  Leg 10: 16 → 21 cargo 271000 — ✅ Served
  Leg 11: 29 → 26 cargo 271000 — ❌ Not Served
  Leg 12: 22 → 3 cargo 271000 — ✅ Served
  Leg 13: 32 → 20 cargo 252329 — ✅ Served
  Leg 14: 20 → 8 cargo 271000 — ✅ Served
  Leg 15: 10 → 0 cargo 270900 — ✅ Served
  Leg 16: 14 → 26 cargo 270112 — ✅ Served
  Leg 17: 14 → 16 cargo 252329 — ❌ Not Served
  Leg 18: 15 → 31 cargo 271000 — ❌ Not Served
  Leg 19: 9 → 21 cargo 271000 — ✅ Served

🛳️ VESSEL OP

🌍 GLOBAL MODEL SUMMARY
  Revenue:       $32,411,747
  Total Cost:    $14,050,888
  Profit (Obj):  $18,360,858
  Penalties:     $0

📦 LEG ASSIGNMENTS:
  Leg 0: 13 → 12 cargo 252310 — ❌ Not Served
  Leg 1: 3 → 26 cargo 271000 — ✅ Served
  Leg 2: 35 → 1 cargo 252310 — ✅ Served
  Leg 3: 6 → 36 cargo 252329 — ✅ Served
  Leg 4: 4 → 22 cargo 260600 — ✅ Served
  Leg 5: 13 → 36 cargo 271000 — ✅ Served
  Leg 6: 11 → 25 cargo 270900 — ✅ Served
  Leg 7: 14 → 29 cargo 270112 — ✅ Served
  Leg 8: 12 → 39 cargo 270900 — ❌ Not Served
  Leg 9: 13 → 0 cargo 270900 — ✅ Served
  Leg 10: 15 → 1 cargo 252100 — ✅ Served
  Leg 11: 28 → 46 cargo 271000 — ✅ Served
  Leg 12: 17 → 25 cargo 260112 — ✅ Served
  Leg 13: 13 → 4 cargo 252310 — ❌ Not Served
  Leg 14: 26 → 12 cargo 270210 — ✅ Served
  Leg 15: 5 → 12 cargo 271000 — ✅ Served
  Leg 16: 9 → 1 cargo 271000 — ❌ Not Served
  Leg 17: 25 → 35 cargo 270900 — ✅ Served
  Leg 18: 29 → 45 cargo 252329 — ✅ Served
  Leg 19: 27 → 23 cargo 260111 — ✅ Served
  Leg 20: 5 → 

🌍 GLOBAL MODEL SUMMARY
  Revenue:       $41,073,585
  Total Cost:    $21,165,838
  Profit (Obj):  $19,907,746
  Penalties:     $0

📦 LEG ASSIGNMENTS:
  Leg 0: 7 → 15 cargo 270900 — ✅ Served
  Leg 1: 32 → 39 cargo 252310 — ✅ Served
  Leg 2: 35 → 46 cargo 271000 — ✅ Served
  Leg 3: 23 → 24 cargo 271000 — ✅ Served
  Leg 4: 29 → 22 cargo 271000 — ❌ Not Served
  Leg 5: 22 → 3 cargo 271000 — ❌ Not Served
  Leg 6: 3 → 12 cargo 271000 — ✅ Served
  Leg 7: 6 → 25 cargo 252329 — ✅ Served
  Leg 8: 35 → 25 cargo 260112 — ✅ Served
  Leg 9: 22 → 24 cargo 271000 — ✅ Served
  Leg 10: 17 → 12 cargo 260111 — ✅ Served
  Leg 11: 17 → 1 cargo 252100 — ✅ Served
  Leg 12: 28 → 35 cargo 270900 — ✅ Served
  Leg 13: 22 → 31 cargo 271000 — ✅ Served
  Leg 14: 33 → 5 cargo 260111 — ✅ Served
  Leg 15: 26 → 1 cargo 270210 — ❌ Not Served
  Leg 16: 4 → 10 cargo 251710 — ✅ Served
  Leg 17: 7 → 39 cargo 271000 — ✅ Served
  Leg 18: 0 → 15 cargo 260112 — ❌ Not Served
  Leg 19: 13 → 40 cargo 252329 — ❌ Not Served
  Leg 20: 

🌍 GLOBAL MODEL SUMMARY
  Revenue:       $42,826,133
  Total Cost:    $19,396,262
  Profit (Obj):  $23,429,871
  Penalties:     $0

📦 LEG ASSIGNMENTS:
  Leg 0: 4 → 44 cargo 251710 — ✅ Served
  Leg 1: 1 → 40 cargo 271000 — ✅ Served
  Leg 2: 21 → 39 cargo 270900 — ✅ Served
  Leg 3: 28 → 36 cargo 271000 — ✅ Served
  Leg 4: 15 → 0 cargo 252310 — ✅ Served
  Leg 5: 7 → 26 cargo 271000 — ✅ Served
  Leg 6: 13 → 46 cargo 271000 — ✅ Served
  Leg 7: 11 → 41 cargo 271000 — ✅ Served
  Leg 8: 34 → 5 cargo 270900 — ✅ Served
  Leg 9: 27 → 38 cargo 270119 — ❌ Not Served
  Leg 10: 11 → 5 cargo 270900 — ❌ Not Served
  Leg 11: 13 → 10 cargo 271000 — ✅ Served
  Leg 12: 1 → 38 cargo 271000 — ✅ Served
  Leg 13: 30 → 16 cargo 252310 — ✅ Served
  Leg 14: 0 → 22 cargo 260111 — ✅ Served
  Leg 15: 25 → 3 cargo 252310 — ✅ Served
  Leg 16: 19 → 26 cargo 261800 — ✅ Served
  Leg 17: 6 → 1 cargo 270210 — ❌ Not Served
  Leg 18: 17 → 22 cargo 252100 — ❌ Not Served
  Leg 19: 9 → 0 cargo 271000 — ✅ Served
  Leg 20: 14 → 4 

In [ ]:
fleet_grid_df_1

,vessel,distance_nm,days,CII,EEOI,revenue_usd,scenario
0,C00,45678.36,222.850758,3.806942,6.114956,1.209192e+07,10
1,C00,24405.82,138.641706,3.776148,6.744789,1.040873e+07,20
2,C01,41821.73,159.010873,3.933230,4.471062,9.506604e+06,20
3,C00,57759.48,294.102006,3.867227,5.188720,1.966462e+07,30
4,C01,23954.21,165.044265,3.867991,5.178940,1.274712e+07,30
5,C00,46957.53,300.708596,3.914137,4.655081,1.844561e+07,40
6,C01,80105.62,355.723805,3.912392,4.672756,2.262798e+07,40
7,C00,62952.94,349.796780,3.892545,4.884821,2.364382e+07,50
8,C01,53291.34,263.409258,3.849890,5.422283,2.070599e+07,50


In [ ]:
scenario_kpi_df_1

,n_legs,objective,vessels_bought,ballast_legs,loaded_legs
scenario,,,,,
10,10,4.925532e+06,1,9,9
20,20,9.436124e+06,2,12,15
30,30,1.836086e+07,2,16,23
40,40,1.990775e+07,2,20,30
50,50,2.342987e+07,2,23,33
